In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from ultralytics import YOLO
import sys
#making custom dataset class
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import numpy as np
import torch
 
import torch.nn as nn
import torch.optim as optim
from torch.nn import Module, Sequential, Conv2d, ConvTranspose2d, LeakyReLU, BatchNorm2d, ReLU, Tanh, Sigmoid, BCELoss 
%matplotlib inline
#import sys
#from ..src.pipeline import PredPipe


C:\Users\JaoPC\AppData\Local\Temp\ipykernel_15004\3937769644.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
sys.path.append(r'C:\Repos\TCC-CDIA')

In [3]:
#training gan model in pytorch
from src.gan import DCGAN

In [4]:
# path to the image directory
dir_data  = "C:/Repos/TCC-CDIA/data/processed/outputs/semafaros/standard"
# setting image shape to 64x64
 
# listing out all file names
nm_imgs   = np.sort(os.listdir(dir_data))

In [5]:
nm_imgs

array(['traffic_light_100_original(1).jpg', 'traffic_light_100_original.jpg', 'traffic_light_101_original(1).jpg', ..., 'traffic_light_99_original(1).jpg', 'traffic_light_99_original(2).jpg', 'traffic_light_99_original.jpg'], dtype='<U34')

In [6]:
device = 'cuda:0' if torch.cuda.is_available() == True else 'cpu'
# device = torch.device(dev)

In [7]:
def plot_images(imgs, grid_size = 5):
    """
    imgs: vector containing all the numpy images
    grid_size: 2x2 or 5x5 grid containing images
    """
     
    fig = plt.figure(figsize = (8, 8))
    columns = rows = grid_size
    plt.title("Training Images")
 
    for i in range(1, columns*rows +1):
        plt.axis("off")
        fig.add_subplot(rows, columns, i)
        plt.imshow(imgs[i])
    plt.show()

In [8]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = datasets.ImageFolder(root='C:/Repos/TCC-CDIA/data/processed/outputs/semafaros', transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)


In [9]:
device

'cuda:0'

In [1]:
dcgan = DCGAN(dataloader, device)
dcgan.train(epochs=1000)

In [12]:
# Assuming that `Generator` is the class name of your model
from src.gan import Generator

# Create an instance of the Generator model
model = Generator()

# Load the model weights
model.load_state_dict(torch.load('../models/dcgan/generator_final.pth', map_location=device))

# Move the model to the same device as the input tensor
model = model.to(device)

# Generate random noise as input
noise = torch.randn(64, 100, 1, 1, device=device)

# Generate images
with torch.no_grad():
    generated_images = model(noise)

In [ ]:
# Assuming that `Generator` is the class name of your model
from src.gan import Generator

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Generator().to(device)
model.load_state_dict(torch.load('../models/dcgan/generator_final.pth', map_location=device))
model.eval()

# Number of batches and batch size
num_batches = 3
batch_size = 64

# Initialize tensor to hold generated images
generated_images = torch.empty((0, 3, 64, 64)).to(device)

# Generate images
for _ in range(num_batches):
    # Generate random noise as input
    noise = torch.randn(batch_size, 100, 1, 1, device=device)

    # Generate images
    with torch.no_grad():
        batch_images = model(noise)

    # Append to the tensor of generated images
    generated_images = torch.cat((generated_images, batch_images), 0)

# Now generated_images contains num_batches * batch_size images

In [ ]:
import math

# Assuming generated_images is a tensor of images
generated_images = generated_images.detach().cpu().numpy()

# Normalize images to [0,1] range
generated_images = (generated_images + 1) / 2

# Get the first num_batches * batch_size images
imgs = generated_images[:num_batches * batch_size]

# Number of images per figure
images_per_figure = 64

# Calculate the number of figures needed
num_figures = math.ceil(len(imgs) / images_per_figure)

for fig_num in range(num_figures):
    # Get the images for this figure
    fig_imgs = imgs[fig_num*images_per_figure:(fig_num+1)*images_per_figure]

    # Create a grid of subplots
    fig, axs = plt.subplots(8, 8, figsize=(10, 10))

    for i, img in enumerate(fig_imgs):
        # Calculate subplot position
        row = i // 8
        col = i % 8

        # Plot the image
        axs[row, col].imshow(np.transpose(img, (1, 2, 0)))
        axs[row, col].axis('off')

    plt.tight_layout()

    # Save the figure in the models folder
    plt.savefig(f'../models/generated_images_{fig_num}.png')

    plt.show()

In [ ]:
import torchvision.transforms as transforms
from PIL import Image
import os

# Number of batches and batch size
num_images = 500
batch_size = 64
num_batches = num_images // batch_size + (num_images % batch_size != 0)

# Initialize tensor to hold generated images
generated_images = torch.empty((0, 3, 64, 64)).to(device)

# Generate images
for _ in range(num_batches):
    # Generate random noise as input
    noise = torch.randn(64, 100, 1, 1, device=device)

    # Generate images
    with torch.no_grad():
        batch_images = model(noise)

    # Append to the tensor of generated images
    generated_images = torch.cat((generated_images, batch_images), 0)

# Now generated_images contains num_batches * batch_size images

# Convert tensor to PIL Images and save
to_pil = transforms.ToPILImage()
output_dir = '../data/processed/outputs/gan' 
os.makedirs('../data/processed/outputs/gan', exist_ok=True)

for i, img in enumerate(generated_images):
    # Normalize the image tensor to 0-1 range
    img = (img - img.min()) / (img.max() - img.min())
    pil_img = to_pil(img.cpu())  # Convert to PIL Image
    try:
        pil_img.save(f'{output_dir}/generated_image_{i+1}.jpg')
    except Exception as e:
        print(f"Failed to save image {i+1}: {e}")  # Save the image